# BB SMA Reversion Strategy - Backtest

This notebook demonstrates the backtesting process for the BB SMA Reversion Strategy on VN30F1M futures.

In [ ]:
import json
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Add parent directory to path
sys.path.insert(0, os.path.dirname(os.path.dirname(os.path.abspath('.'))))

from src.data.data_loader import DataLoader
from src.data.data_processor import DataProcessor
from src.strategy.signal_generator import SignalGenerator
from src.backtest.backtest_engine import BacktestEngine
from src.backtest.performance import PerformanceAnalyzer
from src.visualization.backtest import BacktestVisualizer

## 1. Configuration

In [ ]:
# Load configuration
with open('config/strategy_config.json', 'r') as f:
    config = json.load(f)

print("Strategy Configuration:")
print(json.dumps(config['parameters'], indent=2))

## 2. Generate/Load Data

In [ ]:
def generate_dummy_data(start_date='2024-01-01', end_date='2024-06-01', timeframe='15min'):
    """Generate dummy OHLCV data for testing when database is not available"""
    print("Generating dummy data for testing...")

    # Generate date range for trading hours only
    dates = pd.date_range(start=start_date, end=end_date, freq='D')
    trading_dates = [d for d in dates if d.weekday() < 5]  # Only weekdays

    candles = []
    base_price = 1350  # Base price for VN30F1M

    for date in trading_dates:
        # Morning session: 09:15 - 11:30
        for hour in range(9, 12):
            for minute in [0, 15, 30, 45]:
                if hour == 9 and minute < 15:
                    continue

                # Skip lunch break: 11:30 - 13:00
                if hour == 11 and minute > 30:
                    continue

                # Afternoon session: 13:00 - 14:45
                if hour == 14 and minute > 45:
                    continue

                dt = pd.Timestamp(year=date.year, month=date.month, day=date.day,
                                  hour=hour, minute=minute)

                # Generate random price with some mean-reverting tendency
                change = np.random.randn() * 2
                base_price = base_price + change

                open_price = base_price + np.random.randn() * 0.5
                high_price = max(open_price, base_price) + abs(np.random.randn()) * 1
                low_price = min(open_price, base_price) - abs(np.random.randn()) * 1
                close_price = base_price
                volume = np.random.randint(100, 1000)

                candles.append({
                    'datetime': dt,
                    'open': open_price,
                    'high': high_price,
                    'low': low_price,
                    'close': close_price,
                    'volume': volume
                })

    df = pd.DataFrame(candles)
    print(f"Generated {len(df)} candles")
    return df

# Generate dummy data (use real data by setting use_dummy_data=False)
use_dummy_data = True

if use_dummy_data:
    ohlcv_df = generate_dummy_data('2024-01-01', '2024-06-01', '15min')
else:
    # Load from database
    loader = DataLoader()
    tick_data = loader.get_active_contract_data('2024-01-01', '2024-06-01')
    
    processor = DataProcessor()
    ohlcv_df = processor.resample_to_ohlcv(tick_data, timeframe='15min')

ohlcv_df.head(10)

## 3. Add Technical Indicators

In [ ]:
# Get parameters from config
bb_window = config['parameters']['bb_window']
bb_std = config['parameters']['bb_std']

# Add indicators
processor = DataProcessor()
df_with_indicators = processor.add_indicators(ohlcv_df, bb_window=bb_window, bb_std=bb_std)

# Prepare for backtest
df_prepared = processor.prepare_data_for_backtest(df_with_indicators)

print(f"Data with indicators shape: {df_prepared.shape}")
df_with_indicators[['datetime', 'close', 'sma', 'std', 'upper_band', 'lower_band']].tail(10)

## 4. Generate Trading Signals

In [ ]:
# Generate signals
signal_generator = SignalGenerator()
df_signals = signal_generator.generate_signals(df_prepared)

# Drop rows with NaN in key columns
df_signals = df_signals.dropna(subset=['sma', 'lower_band', 'buy_signal'])

print(f"Total signals: {len(df_signals)}")
print(f"Buy signals: {df_signals['buy_signal'].sum()}")

# Show some buy signals
buy_signals = df_signals[df_signals['buy_signal'] == 1][['datetime', 'close', 'sma', 'lower_band', 'buy_signal']]
print("\nBuy Signals:")
buy_signals.head(10)

## 5. Run Backtest

In [ ]:
# Run backtest
backtest_engine = BacktestEngine(
    initial_balance=config['backtest']['initial_capital'],
    commission=config['backtest']['commission'],
    stop_loss_points=config['risk_management']['stop_loss_points']
)

results = backtest_engine.run_backtest(df_signals)

print(f"Final Balance: {results['final_balance']:.2f}")
print(f"Total Return: {results['total_return']:.2%}")
print(f"Total Trades: {len(results['trades'])}")

## 6. Performance Metrics

In [ ]:
# Calculate performance metrics
performance = PerformanceAnalyzer(risk_free_rate=config['backtest']['risk_free_rate'])
metrics = performance.calculate_metrics(
    results['trades'],
    results['portfolio_history'],
    config['backtest']['initial_capital']
)

performance.print_metrics(metrics)

## 7. Visualizations

In [ ]:
# Create visualizer
visualizer = BacktestVisualizer(save_dir='readme_results')

# Plot equity curve
visualizer.plot_equity_curve(
    results['portfolio_history'],
    title='Equity Curve',
    save_path='readme_results/equity_curve.png'
)

In [ ]:
# Plot candlestick with signals
visualizer.plot_candlestick_with_signals(
    df_signals,
    results['trades'],
    title='Backtest Chart with Entry/Exit Signals',
    save_path='readme_results/backtest_chart.png'
)

In [ ]:
# Plot trade distribution
visualizer.plot_trade_distribution(
    results['trades'],
    title='Trade Distribution',
    save_path='readme_results/trade_distribution.png'
)

In [ ]:
# Plot exit analysis
visualizer.plot_exit_analysis(
    results['trades'],
    title='Exit Analysis',
    save_path='readme_results/exit_analysis.png'
)

## 8. Trade Details

In [ ]:
# Show trade details
if not results['trades'].empty:
    print("Trade Details:")
    display(results['trades'])
else:
    print("No trades executed.")

## Summary

This notebook demonstrates the complete backtesting workflow for the BB SMA Reversion Strategy:

1. **Data Generation/Loading**: Generate or load OHLCV data
2. **Indicator Calculation**: Calculate Bollinger Bands and SMA
3. **Signal Generation**: Generate buy signals based on price crossing below lower Bollinger Band
4. **Backtesting**: Run backtest with defined exit rules (TP at SMA, SL at -2 points, market close)
5. **Performance Analysis**: Calculate comprehensive performance metrics
6. **Visualization**: Generate charts for analysis